<a href="https://colab.research.google.com/github/mldurga/projects/blob/master/wagon_shipment_reader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wagon shipment number scanner

## Its skeleton version, further improvement is in pipeline


It is using google drive for storing the HTML file showing all the data with images and Excel file for further downstream tasks. So you need to login with the google credentials and copy paste the all the wagon images collected from the field to the folder name "wagon_images".


In [ ]:
#Download test images if you want to test the application by unhashing and executing following 
!wget https://user-images.githubusercontent.com/19243618/147801374-0915efba-943d-4811-866e-b00fd0a961e4.jpeg 
!wget https://user-images.githubusercontent.com/19243618/147801504-66b17664-ef45-48e1-9f20-fc077d66676a.jpeg 
!wget https://user-images.githubusercontent.com/19243618/147801523-1d33a350-167d-406a-a8aa-4d67d28e3f9b.jpeg 
!wget https://user-images.githubusercontent.com/19243618/147801534-17340205-5600-4e76-9cb1-1b47c0f140b2.jpeg 
!wget https://user-images.githubusercontent.com/19243618/147801565-c3731a50-9eda-430b-97ce-bfb2433e9b5d.jpeg 
!wget https://user-images.githubusercontent.com/19243618/147801590-dce11c1a-d8bd-47f9-aa43-28f8ce471d4e.jpeg 
!wget https://user-images.githubusercontent.com/19243618/147801613-1e7b6cbb-24b1-4074-a6a0-c332e40b7b6c.jpeg 
!wget https://user-images.githubusercontent.com/19243618/147801637-921474aa-ce39-4144-afca-e8910c5d1438.jpeg 
!wget https://user-images.githubusercontent.com/19243618/147801681-1eaf53dc-f6b4-4a46-9048-633681a5c0ed.jpeg 
!wget https://user-images.githubusercontent.com/19243618/147801703-452cc054-98c2-40e4-9585-5b356c5ccb5f.jpeg 

In [12]:
!pip install paddlepaddle paddleocr fastai -Uqq

from fastai.vision.all import *
from paddleocr import PaddleOCR
from PIL import Image
import re
import base64
import pandas as pd
from io import BytesIO
from IPython.display import HTML



import IPython
import uuid
from google.colab import output

class InvokeButton(object):
  def __init__(self, title, callback):
    self._title = title
    self._callback = callback

  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)

    template = """<button id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id)
    return html




path="/content/"
fn=get_image_files('/content/')
region_names={'CR', 'KR', 'NR', 'NCR', 'NER', 'NFR', 'NWR', 'ER', 'ECR', 'ECOR',
       'SR', 'SCR', 'SCOR', 'SER', 'SECR', 'SWR', 'WR', 'WCR'}
wagon={'steel_wagon':['BFNS','BOST','BOSTHS','BOXN','BRN','BRNA','BRNAHS'],'brake_van':['BVCM','BVZC','BVZI']}

def wagon_num(num):
  x=[]
  for i in range(len(num)):
    if len(num[i])!=11:
      o=i
      while o<(len(num)-1):
        if len(num[i]+num[o+1])==11:
          x.append(num[i]+num[o+1])
        o+=1
    else:x.append(num[i])
  return x 

def rail_region(txts):
  region_name=[]
  if len(txts)>0:
    for x in txts:
      if x in region_names:
        region_name.append(x)
    return region_name

def get_thumbnail(path):
    i = Image.open(path)
    i.thumbnail((300, 200), Image.LANCZOS)
    return i

def image_base64(im):
    if isinstance(im, str):
        im = get_thumbnail(im)
    with BytesIO() as buffer:
        im.save(buffer, 'jpeg')
        return base64.b64encode(buffer.getvalue()).decode()

def image_formatter(im):
    return f'<img src="data:image/jpeg;base64,{image_base64(im)}">'

pd.set_option('display.max_colwidth', -1)


def predict():
   wagon_number=[];region=[];texts=[];nums=[];image=[]
   for i in fn:
     ocr = PaddleOCR(use_angle_cls=True, lang='en',use_gpu=False)
     result = ocr.ocr(str(i), cls=True)
     outs = [line[1][0] for line in result]
     txts=[item.upper() for sublist in [re.findall("[a-zA-Z]+",o) for o in outs] for item in sublist]
     num=[item for sublist in [re.findall(r'\d+',o) for o in outs] for item in sublist]
     r_region=rail_region(txts)
     wagon=wagon_num(num)
     wagon_number.append({wagon[0] if len(wagon)>0 else None});region.append(r_region);texts.append(txts);nums.append(num);image.append(str(i))
   df=pd.DataFrame({'wagon_number':wagon_number, 'region':region, 'texts':texts,'nums':nums,'image':image })
   df.to_excel(r'/content/wagon_excel.xlsx', index=False)
   return HTML(df.to_html(formatters={'image': image_formatter}, escape=False))

InvokeButton('Predict', predict)